# Description of DataFrames
### realtor_data_df 
- DataFrame directly read from the realtor_data.csv file

### clean_df
- Contains just the headers/ columns needed for our analysis
- Dropped all null values in any of those columns
- Contains data for all states in data set

### ct_df, ny_df, ma_df, nj_df
- Contains just data for the 4 states being investigated from the clean_df

### states_df
- Combines the four DataFrames described above

### q1_df
- This is the name of the DataFrame required in the find_outliers function
- This DataFrame needs to be set to the correct DataFrame before running the function

### clean_ct_df, clean_ny_df, clean_ma_df, clean_nj_df
- Each DataFrame has the state-specific outliers removed for each column

### clean_states_df
- Combines the four clean DataFrames described above



In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint

In [2]:
realtor_data = Path('realtor_data.csv') 
realtor_data_df = pd.read_csv(realtor_data)

realtor_data_df.head()

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000.0,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [3]:
realtor_data_df = realtor_data_df[['price', 'bed', 'bath', 'acre_lot', 'house_size', 'state', 'city']]
clean_df = realtor_data_df.dropna()
clean_df.head()

,price,bed,bath,acre_lot,house_size,state,city
0,105000.0,3.0,2.0,0.12,920.0,Puerto Rico,Adjuntas
1,80000.0,4.0,2.0,0.08,1527.0,Puerto Rico,Adjuntas
2,67000.0,2.0,1.0,0.15,748.0,Puerto Rico,Juana Diaz
3,145000.0,4.0,2.0,0.10,1800.0,Puerto Rico,Ponce
5,179000.0,4.0,3.0,0.46,2520.0,Puerto Rico,San Sebastian


In [16]:
# Filter out just Connecticut, New York, Massechusettes, and New Jersey
ct_df = clean_df[clean_df['state'] == 'Connecticut']
ny_df = clean_df[clean_df['state'] == 'New York']
ma_df = clean_df[clean_df['state'] == 'Massachusetts']
nj_df = clean_df[clean_df['state'] == 'New Jersey']

df_list = [ct_df, ny_df, ma_df, nj_df]

states_df = pd.concat(df_list)
states_df.head()


,price,bed,bath,acre_lot,house_size,state,city
27820,225000.0,3.0,1.0,3.93,1572.0,Connecticut,Willington
27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester


In [5]:
# Funtion that identifies lower quartile, upper quartile, IQR, outliers, and number of outliers
# The data frame that q1_df needs to be defined before the function is run
# For each state, set q1_df to be a copy of the state DataFrame (i.e. ny_df)
q1_df = ct_df
variable_list = ['price', 'bed', 'bath', 'acre_lot', 'house_size']

# variable is the column title
def find_outliers(variable):
    
    # Creates a list/series from the single column that comes from the variable
    feature = q1_df[variable]
    
    # Define quartile, and set values for lower q, upper q, and IQR
    quartiles = feature.quantile([.25,.5,.75])
    lowerq = quartiles[.25]
    upperq = quartiles[.75]
    iqr = upperq-lowerq
    
    # Creates an empty list for outliers
    outliers = []
    
    # Loops through the data to identify any values that are outliers and adds them to the list
    for x in feature:
        if (x > (upperq + 1.5*iqr)) | (x < (lowerq - 1.5*iqr)):
            outliers.append(x)

    # Prints an output that gives the data we're looking for
    print(f'{variable.title()}')
    print(f'\tLower Quartile: {lowerq}')
    print(f'\tUpper Quartile: {upperq}')
    print(f'\tOutliers are less than {lowerq - 1.5*iqr}, and greater than {upperq + 1.5*iqr}.')
    print(f'\tThere are {len(outliers)} outliers.')

In [6]:
# Identify outliers for CT

q1_df = ct_df
for variable in variable_list:
    find_outliers(variable)


Price
	Lower Quartile: 239000.0
	Upper Quartile: 479900.0
	Outliers are less than -122350.0, and greater than 841250.0.
	There are 5855 outliers.
Bed
	Lower Quartile: 3.0
	Upper Quartile: 4.0
	Outliers are less than 1.5, and greater than 5.5.
	There are 6757 outliers.
Bath
	Lower Quartile: 2.0
	Upper Quartile: 3.0
	Outliers are less than 0.5, and greater than 4.5.
	There are 4337 outliers.
Acre_Lot
	Lower Quartile: 0.21
	Upper Quartile: 1.12
	Outliers are less than -1.1550000000000002, and greater than 2.4850000000000003.
	There are 7671 outliers.
House_Size
	Lower Quartile: 1306.0
	Upper Quartile: 2591.0
	Outliers are less than -621.5, and greater than 4518.5.
	There are 2902 outliers.


In [17]:
# Remove outliers for CT

clean_ct_df = ct_df[(ct_df['price'] < 841250) &
              (ct_df['bed'] < 5.5) &
              (ct_df['bath'] < 4.5) &
              (ct_df['acre_lot'] < 2.4850000000000003) &
              (ct_df['house_size'] < 4518.5)
              ]
clean_ct_df

,price,bed,bath,acre_lot,house_size,state,city
27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
27830,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor
...,...,...,...,...,...,...,...
464216,755000.0,4.0,3.0,1.60,2586.0,Connecticut,Stamford
464217,799000.0,4.0,3.0,1.52,3154.0,Connecticut,Stamford
464222,799000.0,4.0,4.0,1.19,3128.0,Connecticut,Stamford
464224,699000.0,4.0,3.0,1.04,2156.0,Connecticut,Stamford


In [8]:
# Identify outliers for NY

q1_df = ny_df
for variable in variable_list:
    find_outliers(variable)

Price
	Lower Quartile: 482000.0
	Upper Quartile: 1250000.0
	Outliers are less than -670000.0, and greater than 2402000.0.
	There are 1801 outliers.
Bed
	Lower Quartile: 3.0
	Upper Quartile: 5.0
	Outliers are less than 0.0, and greater than 8.0.
	There are 634 outliers.
Bath
	Lower Quartile: 2.0
	Upper Quartile: 3.0
	Outliers are less than 0.5, and greater than 4.5.
	There are 2253 outliers.
Acre_Lot
	Lower Quartile: 0.05
	Upper Quartile: 0.49
	Outliers are less than -0.61, and greater than 1.15.
	There are 2657 outliers.
House_Size
	Lower Quartile: 1320.0
	Upper Quartile: 2750.0
	Outliers are less than -825.0, and greater than 4895.0.
	There are 1120 outliers.


In [19]:
# Remove outliers for NY

clean_ny_df = ny_df[(ny_df['price'] < 2402000) &
                    (ny_df['bed'] < 8.0) &
                    (ny_df['bath'] < 4.5) &
                    (ny_df['acre_lot'] < 1.15) &
                    (ny_df['house_size'] < 4895)
                    ]
clean_ny_df

,price,bed,bath,acre_lot,house_size,state,city
54258,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
54530,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
54531,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
54539,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
54542,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...
732127,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
732237,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
732292,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
732295,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [10]:
# Identify outliers for MA

q1_df = ma_df
for variable in variable_list:
    find_outliers(variable)

Price
	Lower Quartile: 425000.0
	Upper Quartile: 1050000.0
	Outliers are less than -512500.0, and greater than 1987500.0.
	There are 10940 outliers.
Bed
	Lower Quartile: 3.0
	Upper Quartile: 4.0
	Outliers are less than 1.5, and greater than 5.5.
	There are 19507 outliers.
Bath
	Lower Quartile: 2.0
	Upper Quartile: 3.0
	Outliers are less than 0.5, and greater than 4.5.
	There are 12348 outliers.
Acre_Lot
	Lower Quartile: 0.13
	Upper Quartile: 0.7
	Outliers are less than -0.725, and greater than 1.555.
	There are 11859 outliers.
House_Size
	Lower Quartile: 1422.0
	Upper Quartile: 3100.0
	Outliers are less than -1095.0, and greater than 5617.0.
	There are 5365 outliers.


In [20]:
# Remove outliers for MA

clean_ma_df = ma_df[(ma_df['price'] < 1987500) & 
                    (ma_df['bed'] < 5.5) &
                    (ma_df['bath'] < 4.5) &
                    (ma_df['acre_lot'] < 1.555) &
                    (ma_df['house_size'] < 5617.5)
]

clean_ma_df

,price,bed,bath,acre_lot,house_size,state,city
24231,180000.0,2.0,1.0,0.34,676.0,Massachusetts,Agawam
24236,239900.0,3.0,1.0,0.46,1196.0,Massachusetts,Agawam
24237,525000.0,3.0,3.0,0.45,2314.0,Massachusetts,Agawam
24238,289900.0,3.0,2.0,0.36,1276.0,Massachusetts,Agawam
24241,275000.0,4.0,2.0,0.11,1732.0,Massachusetts,Agawam
...,...,...,...,...,...,...,...
460678,409000.0,3.0,2.0,1.25,1792.0,Massachusetts,Otis
460694,709000.0,2.0,2.0,0.37,1952.0,Massachusetts,Otis
460714,175000.0,4.0,1.0,0.13,1554.0,Massachusetts,Great Barrington
460716,159900.0,2.0,1.0,0.06,800.0,Massachusetts,Otis


In [12]:
# Identify outliers for NJ

q1_df = nj_df
for variable in variable_list:
    find_outliers(variable)

Price
	Lower Quartile: 275000.0
	Upper Quartile: 625000.0
	Outliers are less than -250000.0, and greater than 1150000.0.
	There are 5685 outliers.
Bed
	Lower Quartile: 3.0
	Upper Quartile: 4.0
	Outliers are less than 1.5, and greater than 5.5.
	There are 5842 outliers.
Bath
	Lower Quartile: 2.0
	Upper Quartile: 3.0
	Outliers are less than 0.5, and greater than 4.5.
	There are 5157 outliers.
Acre_Lot
	Lower Quartile: 0.11
	Upper Quartile: 0.46
	Outliers are less than -0.41500000000000004, and greater than 0.9850000000000001.
	There are 9966 outliers.
House_Size
	Lower Quartile: 1365.0
	Upper Quartile: 2600.0
	Outliers are less than -487.5, and greater than 4452.5.
	There are 4978 outliers.


In [21]:
# Remove outliers for NJ

clean_nj_df = nj_df[(nj_df['price'] < 1150000) &
                    (nj_df['bed'] < 5.5) &
                    (nj_df['bath'] < 4.5) &
                    (nj_df['acre_lot'] < .9850000000000001) &
                    (nj_df['house_size'] < 4452.5)
                    ]

clean_nj_df

,price,bed,bath,acre_lot,house_size,state,city
30126,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
45320,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
385082,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
388797,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
389915,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
...,...,...,...,...,...,...,...
733133,475000.0,4.0,3.0,0.23,2252.0,New Jersey,South Brunswick
733136,350000.0,2.0,2.0,0.01,1551.0,New Jersey,Piscataway
733146,241000.0,2.0,2.0,0.50,1077.0,New Jersey,North Brunswick
733147,629000.0,5.0,2.0,0.06,2084.0,New Jersey,New Brunswick


In [22]:
clean_df_list = [clean_ct_df, clean_ny_df, clean_ma_df, clean_nj_df]

clean_states_df = pd.concat(clean_df_list)
clean_states_df.head()

,price,bed,bath,acre_lot,house_size,state,city
27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
27830,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor


In [23]:
clean_states_df.to_csv('clean_states.csv')